# Pipeline built-in factors

本文將介紹 zipline 內建的因子。

In [ ]:
import pandas as pd 
import numpy as np 
import os 
os.environ['TEJAPI_KEY'] = "your key" 
os.environ['TEJAPI_BASE'] = "http://api.tej.com.tw"
os.environ['mdate'] = '20150101 20230702'
os.environ['ticker'] = "2330 IR0001"
!zipline ingest -b tquant

[2023-09-01 05:27:47.602922] INFO: zipline.data.bundles.core: Ingesting tquant.
Merging daily equity files:  [####################################]   


<span id="menu"></span>
## 選單
* [AverageDollarVolume](#AverageDollarVolume)
* [BollingerBands](#BollingerBands)
* [DailyReturns](#DailyReturns)
* [SimpleMovingAverage](#SimpleMovingAverage)
* [LinearWeightedMovingAverage](#LinearWeightedMovingAverage)
* [ExponentialWeightedMovingAverage](#ExponentialWeightedMovingAverage)
* [ExponentialWeightedMovingStdDev](#ExponentialWeightedMovingStdDev)
* [Latest](#Latest)
* [MaxDrawdown](#MaxDrawdown) 
* [Returns](#Returns)
* [RollingPearson](#RollingPearson)
* [RollingLinearRegressionOfReturns](#RollingLinearRegressionOfReturns)
* [RollingSpearmanOfReturns](#RollingSpearmanOfReturns)
* [SimpleBeta](#SimpleBeta)
* [RSI](#RSI)
* [VWAP](#VWAP)
* [WeightedAverageValue](#WeightedAverageValue)
* [PercentChange](#PercentChange)
* [PeerCount](#PeerCount)
* [RateOfChangePercentage](#RateOfChangePercentage)
* [Aroon](#Aroon)
* [FastStochasticOscillator](#FastStochasticOscillator)
* [TrueRange](#TrueRange)
* [IchimokuKinkoHyo](#IchimokuKinkoHyo)

In [2]:
from zipline.master import run_pipeline    
from zipline.pipeline import Pipeline
from zipline.pipeline.data import TWEquityPricing
start_time = pd.Timestamp("2018-02-02", tz="UTC")
end_time = pd.Timestamp("2022-07-02", tz="UTC")

<span id="AverageDollarVolume"></span>
### zipline.pipeline.factors.AverageDollarVolume

計算當日以前 n 天的資產平均價值，$ 每日平均價值 = 成交量 \times 價格$。  

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_, optional
        所欲計算之價格資料，預設 = EquityPricing.close。
* window_length: _int_
        決定 n 天。
        
[回到選單](#menu)

In [3]:
from zipline.pipeline.factors import AverageDollarVolume
def make_pipeline():
    return Pipeline(
        columns = {
            "avg_dollar_volume": AverageDollarVolume(
                inputs = [TWEquityPricing.close, TWEquityPricing.volume],
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

avg_dollar_volume
2018-02-02 00:00:00+00:00 Equity(0 [2330])         1.065447e+10
                          Equity(1 [IR0001])       4.975421e+13
2018-02-05 00:00:00+00:00 Equity(0 [2330])         9.914759e+09
                          Equity(1 [IR0001])       4.839457e+13
2018-02-06 00:00:00+00:00 Equity(0 [2330])         9.892107e+09
...                                                         ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])       7.144668e+13
2022-06-30 00:00:00+00:00 Equity(0 [2330])         1.734140e+10
                          Equity(1 [IR0001])       7.240678e+13
2022-07-01 00:00:00+00:00 Equity(0 [2330])         1.809191e+10
                          Equity(1 [IR0001])       7.315819e+13

[2146 rows x 1 columns]

<span id="BollingerBands"></span>
### zipline.pipeline.factors.BollingerBands

布林通道利用統計學概念，創造上、中、下三軌，中軌為 n 日的移動平均線，上下兩軌分別為中軌 $\pm m$ 個標準差。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_, optional
        所欲計算之價格資料，預設 = EquityPricing.close。
* window_length: _int_
        決定 n 天。
* k: _int_
        決定 m 個標準差。
        
BollingerBand 提供兩種呼叫方法，請見下方程式碼示範:

[回到選單](#menu)

In [4]:
# 方法一
from zipline.pipeline.factors import BollingerBands
def make_pipeline():
    return Pipeline(
        columns = {
            "bbands": BollingerBands(
                inputs = [TWEquityPricing.close], 
                window_length = 14,
                k=1.5
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

bbands
2018-02-02 00:00:00+00:00 Equity(0 [2330])    (242.03280256301392, 253.64285714285714, 265.2...
                          Equity(1 [IR0001])  (18825.27804936642, 19052.489999999998, 19279....
2018-02-05 00:00:00+00:00 Equity(0 [2330])    (244.7387059773734, 255.03571428571428, 265.33...
                          Equity(1 [IR0001])  (18878.33432106565, 19073.285, 19268.23567893435)
2018-02-06 00:00:00+00:00 Equity(0 [2330])    (247.50575256841418, 255.92857142857142, 264.3...
...                                                                                         ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  (30313.75941998411, 31462.749285714282, 32611....
2022-06-30 00:00:00+00:00 Equity(0 [2330])    (485.4329390766828, 501.5947937131631, 517.756...
                          Equity(1 [IR0001])  (30335.55527873807, 31296.089285714283, 32256....
2022-07-01 00:00:00+00:00 Equity(0 [2330])    (482.723018720442, 497.94218355318554, 513.161...
                          Equity(1 [IR0001])  (30213.77504424966, 31095.97214285714, 31978.1...

[2146 rows x 1 columns]

In [5]:
# 方法二
from zipline.pipeline.factors import BollingerBands
def make_pipeline():
    upper = BollingerBands(inputs = [TWEquityPricing.close], window_length = 14, k = 1.5).upper
    middle = BollingerBands(inputs = [TWEquityPricing.close], window_length = 14, k = 1.5).middle
    lower = BollingerBands(inputs = [TWEquityPricing.close], window_length = 14, k = 1.5).lower
    return Pipeline(
        columns = {
            "upper_bound":upper,
            "middle":middle,
            "lower_bound":lower
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

upper_bound        middle  \
2018-02-02 00:00:00+00:00 Equity(0 [2330])      265.252912    253.642857   
                          Equity(1 [IR0001])  19279.701951  19052.490000   
2018-02-05 00:00:00+00:00 Equity(0 [2330])      265.332723    255.035714   
                          Equity(1 [IR0001])  19268.235679  19073.285000   
2018-02-06 00:00:00+00:00 Equity(0 [2330])      264.351390    255.928571   
...                                                    ...           ...   
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  32611.739151  31462.749286   
2022-06-30 00:00:00+00:00 Equity(0 [2330])      517.756648    501.594794   
                          Equity(1 [IR0001])  32256.623293  31296.089286   
2022-07-01 00:00:00+00:00 Equity(0 [2330])      513.161348    497.942184   
                          Equity(1 [IR0001])  31978.169241  31095.972143   

                                               lower_bound  
2018-02-02 00:00:00+00:00 Equity(0 [2330])      242.032803  
                          Equity(1 [IR0001])  18825.278049  
2018-02-05 00:00:00+00:00 Equity(0 [2330])      244.738706  
                          Equity(1 [IR0001])  18878.334321  
2018-02-06 00:00:00+00:00 Equity(0 [2330])      247.505753  
...                                                    ...  
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  30313.759420  
2022-06-30 00:00:00+00:00 Equity(0 [2330])      485.432939  
                          Equity(1 [IR0001])  30335.555279  
2022-07-01 00:00:00+00:00 Equity(0 [2330])      482.723019  
                          Equity(1 [IR0001])  30213.775044  

[2146 rows x 3 columns]

<span id="DailyReturns"></span>
    
### zipline.pipeline.factors.DailyReturns

計算日報酬率。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_, optional
        所欲計算之價格資料，預設 = EquityPricing.close。
        
[回到選單](#menu)

In [6]:
from zipline.pipeline.factors import DailyReturns
def make_pipeline():
    return Pipeline(
        columns = {
            "Daily Return": DailyReturns(
                inputs = [TWEquityPricing.close]
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

Daily Return
2018-02-02 00:00:00+00:00 Equity(0 [2330])        0.017647
                          Equity(1 [IR0001])      0.005086
2018-02-05 00:00:00+00:00 Equity(0 [2330])        0.000000
                          Equity(1 [IR0001])     -0.003049
2018-02-06 00:00:00+00:00 Equity(0 [2330])       -0.025048
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])     -0.005615
2022-06-30 00:00:00+00:00 Equity(0 [2330])       -0.013065
                          Equity(1 [IR0001])     -0.009731
2022-07-01 00:00:00+00:00 Equity(0 [2330])       -0.030550
                          Equity(1 [IR0001])     -0.025612

[2146 rows x 1 columns]

<span id="SimpleMovingAverage"></span>

### zipline.pipeline.factors.SimpleMovingAverage

計算 n 日的簡單移動平均。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。
        
[回到選單](#menu
)

In [7]:
from zipline.pipeline.factors import SimpleMovingAverage
def make_pipeline():
    return Pipeline(
        columns = {
            "SMA": SimpleMovingAverage(
                inputs = [TWEquityPricing.close], 
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

SMA
2018-02-02 00:00:00+00:00 Equity(0 [2330])      258.000
                          Equity(1 [IR0001])  19131.611
2018-02-05 00:00:00+00:00 Equity(0 [2330])      258.400
                          Equity(1 [IR0001])  19127.393
2018-02-06 00:00:00+00:00 Equity(0 [2330])      257.550
...                                                 ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  31072.561
2022-06-30 00:00:00+00:00 Equity(0 [2330])      496.550
                          Equity(1 [IR0001])  30962.418
2022-07-01 00:00:00+00:00 Equity(0 [2330])      493.350
                          Equity(1 [IR0001])  30799.546

[2146 rows x 1 columns]

<span id="LinearWeightedMovingAverage"></span>
### zipline.pipeline.factors.LinearWeightedMovingAverage

計算 n 日的線性加權移動平均，計算方法: 

$$
\frac{\sum_{i=1}^{n} i \times x_{i}}{\sum_{i=1}^{n} i}
$$

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。

[回到選單](#menu)

In [8]:
from zipline.pipeline.factors import LinearWeightedMovingAverage
def make_pipeline():
    return Pipeline(
        columns = {
            "LWMA": LinearWeightedMovingAverage(
                inputs = [TWEquityPricing.close], 
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

LWMA
2018-02-02 00:00:00+00:00 Equity(0 [2330])      257.309091
                          Equity(1 [IR0001])  19108.238000
2018-02-05 00:00:00+00:00 Equity(0 [2330])      257.581818
                          Equity(1 [IR0001])  19095.746909
2018-02-06 00:00:00+00:00 Equity(0 [2330])      256.600000
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  30947.352545
2022-06-30 00:00:00+00:00 Equity(0 [2330])      494.372727
                          Equity(1 [IR0001])  30887.412364
2022-07-01 00:00:00+00:00 Equity(0 [2330])      490.636364
                          Equity(1 [IR0001])  30704.338182

[2146 rows x 1 columns]

<span id="ExponentialWeightedMovingAverage"></span>
### zipline.pipeline.factors.ExponentialWeightedMovingAverage

計算 n 日的指數加權移動平均，計算方法: 

$$
\frac{\sum_{i=1}^{n} {decay}^i \times x_{i}}{\sum_{i=1}^{n} {decay}^i}
$$

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。
* decay_rate: _float_
        指數衰退率。

[回到選單](#menu)

In [9]:
from zipline.pipeline.factors import ExponentialWeightedMovingAverage
def make_pipeline():
    return Pipeline(
        columns = {
            "EWMA": ExponentialWeightedMovingAverage(
                inputs = [TWEquityPricing.close],
                window_length = 10,
                decay_rate = 0.1
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

EWMA
2018-02-02 00:00:00+00:00 Equity(0 [2330])      259.035181
                          Equity(1 [IR0001])  19111.308296
2018-02-05 00:00:00+00:00 Equity(0 [2330])      259.453518
                          Equity(1 [IR0001])  19067.749830
2018-02-06 00:00:00+00:00 Equity(0 [2330])      253.645352
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  31056.864215
2022-06-30 00:00:00+00:00 Equity(0 [2330])      491.648000
                          Equity(1 [IR0001])  30774.287421
2022-07-01 00:00:00+00:00 Equity(0 [2330])      477.564800
                          Equity(1 [IR0001])  30037.387742

[2146 rows x 1 columns]

<span id="ExponentialWeightedMovingStdDev"></span>
### zipline.pipeline.factors.ExponentialWeightedMovingStdDev

計算 n 日的指數加權移動標準差。
#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。
* window_length: _int_
        決定 n 日。
* decay_rate: _float_
        指數衰退率。

[回到選單](#menu)

In [10]:
from zipline.pipeline.factors import ExponentialWeightedMovingStdDev
def make_pipeline():
    return Pipeline(
        columns = {
            "EWMSTD": ExponentialWeightedMovingStdDev(
                inputs = [TWEquityPricing.close],
                window_length = 10,
                decay_rate = 0.1
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

EWMSTD
2018-02-02 00:00:00+00:00 Equity(0 [2330])      3.307859
                          Equity(1 [IR0001])   71.847546
2018-02-05 00:00:00+00:00 Equity(0 [2330])      1.095966
                          Equity(1 [IR0001])   40.935224
2018-02-06 00:00:00+00:00 Equity(0 [2330])      4.553660
...                                                  ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  152.145031
2022-06-30 00:00:00+00:00 Equity(0 [2330])      4.637399
                          Equity(1 [IR0001])  226.079224
2022-07-01 00:00:00+00:00 Equity(0 [2330])     11.106583
                          Equity(1 [IR0001])  580.480345

[2146 rows x 1 columns]

<span id="Latest"></span>

### zipline.pipeline.factors.Latest

輸出最近一期資料。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        所欲計算之資料。

[回到選單](#menu)

In [11]:
from zipline.pipeline.factors import Latest
def make_pipeline():
    return Pipeline(
        columns = {
            "Latest": Latest(
                inputs = [TWEquityPricing.close]
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

Latest
2018-02-02 00:00:00+00:00 Equity(0 [2330])      259.50
                          Equity(1 [IR0001])  19121.21
2018-02-05 00:00:00+00:00 Equity(0 [2330])      259.50
                          Equity(1 [IR0001])  19062.91
2018-02-06 00:00:00+00:00 Equity(0 [2330])      253.00
...                                                ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  31044.99
2022-06-30 00:00:00+00:00 Equity(0 [2330])      491.00
                          Equity(1 [IR0001])  30742.89
2022-07-01 00:00:00+00:00 Equity(0 [2330])      476.00
                          Equity(1 [IR0001])  29955.51

[2146 rows x 1 columns]

<span id="MaxDrawdown"></span>

### zipline.pipeline.factors.MaxDrawdown

以 n 天為周期計算最大回撤。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算最大回徹所需要的價格資訊。
* window_lengthL: _int_
        以 n 天為週期計算。


[回到選單](#menu)

In [12]:
from zipline.pipeline.factors import MaxDrawdown
def make_pipeline():
    return Pipeline(
        columns = {
            "MaxDrawdown": MaxDrawdown(
                inputs = [TWEquityPricing.close],
                window_length = 1
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

MaxDrawdown
2018-02-02 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0
2018-02-05 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0
2018-02-06 00:00:00+00:00 Equity(0 [2330])            0.0
...                                                   ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])          0.0
2022-06-30 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0
2022-07-01 00:00:00+00:00 Equity(0 [2330])            0.0
                          Equity(1 [IR0001])          0.0

[2146 rows x 1 columns]

<span id="Returns"></span>

### zipline.pipeline.factors.Returns

以 n 天為窗格，計算報酬率。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的價格資訊，預設 = EquityPricing.close。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [13]:
from zipline.pipeline.factors import Returns
def make_pipeline():
    return Pipeline(
        columns = {
            "Returns": Returns(
                inputs = [TWEquityPricing.close],
                window_length = 2
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

Returns
2018-02-02 00:00:00+00:00 Equity(0 [2330])    0.017647
                          Equity(1 [IR0001])  0.005086
2018-02-05 00:00:00+00:00 Equity(0 [2330])    0.000000
                          Equity(1 [IR0001]) -0.003049
2018-02-06 00:00:00+00:00 Equity(0 [2330])   -0.025048
...                                                ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001]) -0.005615
2022-06-30 00:00:00+00:00 Equity(0 [2330])   -0.013065
                          Equity(1 [IR0001]) -0.009731
2022-07-01 00:00:00+00:00 Equity(0 [2330])   -0.030550
                          Equity(1 [IR0001]) -0.025612

[2146 rows x 1 columns]

<span id="RollingPearson"></span>

### zipline.pipeline.factors.RollingPearson

計算因子或變數之間的滾動皮爾森相關係數。

#### Parameters:
* base_factor: _zipline.pipeline.Factor_
        計算相關係數的因子。
* target: _zipline.pipeline.Term or _numerical term_
        與 base_factor 計算彼此間的相關係數。
* correlation_length : _int_
        向前 n 天為窗格計算相關係數，若僅計算前一日設定為 2。
* mask: _zipline.pipeline.Filter_
        遮蔽特定證券所需的濾網，遮蔽者將不會計算相關係數。


[回到選單](#menu)

In [14]:
from zipline.pipeline.factors import RollingPearson
from zipline.pipeline.factors import DailyReturns
from zipline.pipeline.factors import Returns
def make_pipeline():
    base_factor = DailyReturns(inputs = [TWEquityPricing.close])
    target = Returns(inputs = [TWEquityPricing.close], window_length = 6)
    return Pipeline(
        columns = {
            "RollingPearson": RollingPearson(
                base_factor = base_factor,
                target = target,
                correlation_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

RollingPearson
2018-02-02 00:00:00+00:00 Equity(0 [2330])          0.396933
                          Equity(1 [IR0001])        0.476625
2018-02-05 00:00:00+00:00 Equity(0 [2330])          0.306692
                          Equity(1 [IR0001])        0.412337
2018-02-06 00:00:00+00:00 Equity(0 [2330])          0.235285
...                                                      ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])        0.724798
2022-06-30 00:00:00+00:00 Equity(0 [2330])          0.450101
                          Equity(1 [IR0001])        0.612251
2022-07-01 00:00:00+00:00 Equity(0 [2330])          0.520262
                          Equity(1 [IR0001])        0.519533

[2146 rows x 1 columns]

<span id="RollingLinearRegressionOfReturns"></span>

### zipline.pipeline.factors.RollingLinearRegressionOfReturns

以一個特定資產為自變數，其餘資產為應變數，進行 OLS 迴歸。會輸出迴歸式的 beta, alpha, r_value, p_value 與 standard error。迴歸式如下: 

$$
Return_{i,t} = \beta_{i} \times {Certain Return}_{i,t} + \alpha_{i}
$$

#### Parameters:
* target: _zipline.assets.Asset_
        歸類於自變數的資產。
* returns_length : _int_
        向前 n 天為窗格計算報酬，若為日報酬率，設定為 2。
* regression_length: _int_
        計算各項迴歸式的向前窗格天數。
* mask: _zipline.pipeline.Filter_
        遮蔽特定證券所需的濾網，遮蔽者將不會計算相關係數。

[回到選單](#menu)

In [15]:
from zipline.pipeline.factors import RollingLinearRegressionOfReturns
from zipline.pipeline.filters import StaticAssets
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    regressor = RollingLinearRegressionOfReturns(
                target = symbol("IR0001"),
                returns_length = 2,
                regression_length = 14,
            )
    return Pipeline(
        columns = {
            "beta": regressor.beta,
            "alpha": regressor.alpha,
            "r_value": regressor.r_value,
            "p_value": regressor.p_value,
            "stderr": regressor.stderr
        }, screen = ~StaticAssets([symbol("IR0001")])
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(f"Beta: {pipe.beta}")
    print(f"alpha: {pipe.alpha}")
    print(f"r_value: {pipe.r_value}")
    print(f"p_value: {pipe.p_value}")
    print(f"stderr: {pipe.stderr}")

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start_time,
    end = end_time,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


Beta: Equity(0 [2330])    2.628594
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001886
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.911471
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000006
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.342465
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.652884
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.002775
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.914045
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000005
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.339837
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.361614
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.00435
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.923077
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000003
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.284059
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    1.301623
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.00151
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.821674
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000312
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.260638
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.247377
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001036
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.805744
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000503
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.264682
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.272673
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.000861
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.81474
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000387
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.26146
Name: stderr, dtype: float64
Beta: Equity(0 [233

Beta: Equity(0 [2330])    2.280188
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002375
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.794036
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000696
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.503909
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.359463
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001482
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.859279
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000082
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.405453
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.372879
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.001837
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.854886
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000098
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.41571
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    2.06337
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001713
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.885671
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000025
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.312266
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    2.106281
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.002259
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.897298
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000014
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.299121
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.967031
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.002313
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.868266
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000056
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.324442
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    1.015805
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.0004
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.896594
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000014
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.144839
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    0.978157
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.00051
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.911158
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000006
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.127698
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    0.983021
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000801
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.912062
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000006
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.127581
Name: stderr, dtype: float64
Beta: Equity(0 [233

Beta: Equity(0 [2330])    1.04668
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.003629
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.720765
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.003633
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.290587
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    0.895999
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.006841
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.635574
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.014574
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.314189
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    0.937602
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.006103
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.644306
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.012877
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.321267
Name: stderr, dtype: float64
Beta: Equity(0 [2

Beta: Equity(0 [2330])    1.934061
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000605
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.940047
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    5.887201e-07
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.202555
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.834027
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000295
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.932263
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000001
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.205458
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.691023
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.000423
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.917173
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000004
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.212091
Name: stderr, dtype: float64
Beta: Equity

Beta: Equity(0 [2330])    1.392436
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.00321
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.960923
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    4.724468e-08
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.115794
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.364073
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002891
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.942596
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    4.561741e-07
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.139504
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.428374
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.003646
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.879082
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000035
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.223584
Name: stderr, dtype: float64
Beta: Equ

Beta: Equity(0 [2330])    1.085294
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.003459
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.807365
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.00048
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.228969
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.162488
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.002911
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.746347
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.00217
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.299255
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.233842
Name: beta, dtype: float64
alpha: Equity(0 [2330])    0.001667
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.744772
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.002243
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.319138
Name: stderr, dtype: float64
Beta: Equity(0 [23

Beta: Equity(0 [2330])    1.165595
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002452
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.785227
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000876
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.265337
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    1.004066
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.002789
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.788156
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.000813
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.226345
Name: stderr, dtype: float64
Beta: Equity(0 [2330])    0.995991
Name: beta, dtype: float64
alpha: Equity(0 [2330])   -0.003185
Name: alpha, dtype: float64
r_value: Equity(0 [2330])    0.748237
Name: r_value, dtype: float64
p_value: Equity(0 [2330])    0.002084
Name: p_value, dtype: float64
stderr: Equity(0 [2330])    0.25493
Name: stderr, dtype: float64
Beta: Equity(0 [2

<span id="RollingSpearmanOfReturns"></span>

### zipline.pipeline.factors.RollingSpearmanOfReturns

給定一個特定資產，計算其他資產報酬與特定資產報酬的斯匹爾曼等級相關係數。

#### Parameters:
* target: _zipline.assets.Asset_
        特定資產。
* returns_length : _int_
        向前 n 天為窗格計算報酬，若為日報酬率，設定為 2。
* correlation_length : _int_
        計算相關係數的窗格數。
* mask: _zipline.pipeline.Filter_
        遮蔽特定證券所需的濾網，遮蔽者將不會計算相關係數。

[回到選單](#menu)

In [16]:
from zipline.pipeline.factors import RollingSpearmanOfReturns
from zipline.pipeline.filters import StaticAssets
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    regressor = RollingSpearmanOfReturns(
                target = symbol("IR0001"),
                returns_length = 2,
                correlation_length = 14,
            )
    return Pipeline(
        columns = {
            "RollingSpearmanOfReturns": regressor
        }, screen = ~StaticAssets([symbol("IR0001")])
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(pipe)

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start_time,
    end = end_time,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.859341
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.904291
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.926293
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.926293
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.860287
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.875688
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.880089
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.888889
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.915292
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.920879
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.912088
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.89011
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.894505
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.885714
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.872527
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.872527
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.898901
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.784615
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.762637
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.740659
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.687912
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.512088
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.925275
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.902091
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.904291
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.882289
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.864687
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.864687
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.871288
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.863438
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.894275
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.880803
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.907294
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.630769
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.52967
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.604396
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.626374
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.727473
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.749451
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.793407
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.793407
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.775824
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.850549
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.89011
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.806169
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.828196
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.81498
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.828196
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.869087
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.846154
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.859341
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.868132
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.917492
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.89769
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.937294
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.719472
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.719472
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.481848
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.578022
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.67033
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.551648
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.556044
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.63956
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.30989
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.459341
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.578022
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.507692
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.318681
                  RollingSpearmanOfReturns
Equity(0 [2330])                   0.38022
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.428571
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.525275
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.613187
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.652747
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.705495
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.701099
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.701099
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.696703
                  RollingSpearmanOfReturns
Equity(0 [2

                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.956044
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.982418
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.986813
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.986813
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.964835
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.854945
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.854945
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.859341
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.859341
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.832967
                  RollingSpearmanOfReturns
Equity(0 [2330])                  0.894505
                  RollingSpearmanOfReturns
Equity(0 [2

<span id="SimpleBeta"></span>

### zipline.pipeline.factors.SimpleBeta

給定一個特定資產，以該特定資產為自變數，其他資產為應變數，進行迴歸計算 beta 值，與 RollingLinearRegressionOfReturns 的 beta 等價。

#### Parameters:
* target: _zipline.assets.Asset_
        特定資產。
* regression_length  : _int_
        計算 beta 的窗格數。

[回到選單](#menu)

In [17]:
from zipline.pipeline.factors import SimpleBeta
from zipline.pipeline.filters import StaticAssets
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    Beta = SimpleBeta(
                target = symbol("IR0001"),
                regression_length = 30
            )
    return Pipeline(
        columns = {
            "SimpleBeta": Beta
        }, screen = ~StaticAssets([symbol("IR0001")])
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(pipe)

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start_time,
    end = end_time,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


                  SimpleBeta
Equity(0 [2330])    2.118428
                  SimpleBeta
Equity(0 [2330])    2.104674
                  SimpleBeta
Equity(0 [2330])    1.986685
                  SimpleBeta
Equity(0 [2330])    1.437257
                  SimpleBeta
Equity(0 [2330])    1.373916
                  SimpleBeta
Equity(0 [2330])    1.399631
                  SimpleBeta
Equity(0 [2330])    1.438815
                  SimpleBeta
Equity(0 [2330])     1.44381
                  SimpleBeta
Equity(0 [2330])     1.33807
                  SimpleBeta
Equity(0 [2330])    1.332773
                  SimpleBeta
Equity(0 [2330])    1.343458
                  SimpleBeta
Equity(0 [2330])    1.345029
                  SimpleBeta
Equity(0 [2330])     1.34321
                  SimpleBeta
Equity(0 [2330])    1.348084
                  SimpleBeta
Equity(0 [2330])    1.331513
                  SimpleBeta
Equity(0 [2330])    1.318237
                  SimpleBeta
Equity(0 [2330])    1.364871
              

                  SimpleBeta
Equity(0 [2330])    1.339635
                  SimpleBeta
Equity(0 [2330])    1.348383
                  SimpleBeta
Equity(0 [2330])    1.340434
                  SimpleBeta
Equity(0 [2330])    1.345476
                  SimpleBeta
Equity(0 [2330])    1.341735
                  SimpleBeta
Equity(0 [2330])    1.342752
                  SimpleBeta
Equity(0 [2330])    1.676984
                  SimpleBeta
Equity(0 [2330])    1.654035
                  SimpleBeta
Equity(0 [2330])    1.654995
                  SimpleBeta
Equity(0 [2330])     1.60429
                  SimpleBeta
Equity(0 [2330])    1.580894
                  SimpleBeta
Equity(0 [2330])    1.579332
                  SimpleBeta
Equity(0 [2330])    1.582649
                  SimpleBeta
Equity(0 [2330])    1.614015
                  SimpleBeta
Equity(0 [2330])    1.629587
                  SimpleBeta
Equity(0 [2330])    1.664812
                  SimpleBeta
Equity(0 [2330])    1.544194
              

                  SimpleBeta
Equity(0 [2330])    1.952117
                  SimpleBeta
Equity(0 [2330])    1.957812
                  SimpleBeta
Equity(0 [2330])    1.947413
                  SimpleBeta
Equity(0 [2330])    1.918413
                  SimpleBeta
Equity(0 [2330])    1.901883
                  SimpleBeta
Equity(0 [2330])    1.905071
                  SimpleBeta
Equity(0 [2330])    1.810371
                  SimpleBeta
Equity(0 [2330])     1.80259
                  SimpleBeta
Equity(0 [2330])    1.659189
                  SimpleBeta
Equity(0 [2330])    1.610484
                  SimpleBeta
Equity(0 [2330])    1.596756
                  SimpleBeta
Equity(0 [2330])      1.6028
                  SimpleBeta
Equity(0 [2330])      1.6064
                  SimpleBeta
Equity(0 [2330])    1.572111
                  SimpleBeta
Equity(0 [2330])     1.55954
                  SimpleBeta
Equity(0 [2330])    1.527829
                  SimpleBeta
Equity(0 [2330])    1.498956
              

                  SimpleBeta
Equity(0 [2330])    1.157379
                  SimpleBeta
Equity(0 [2330])    1.311989
                  SimpleBeta
Equity(0 [2330])    1.251725
                  SimpleBeta
Equity(0 [2330])    1.216118
                  SimpleBeta
Equity(0 [2330])    1.232152
                  SimpleBeta
Equity(0 [2330])    1.306329
                  SimpleBeta
Equity(0 [2330])    1.308516
                  SimpleBeta
Equity(0 [2330])     1.27986
                  SimpleBeta
Equity(0 [2330])     1.23214
                  SimpleBeta
Equity(0 [2330])    1.215609
                  SimpleBeta
Equity(0 [2330])    1.249246
                  SimpleBeta
Equity(0 [2330])    1.246887
                  SimpleBeta
Equity(0 [2330])    1.225233
                  SimpleBeta
Equity(0 [2330])    1.249992
                  SimpleBeta
Equity(0 [2330])    1.222879
                  SimpleBeta
Equity(0 [2330])     1.22397
                  SimpleBeta
Equity(0 [2330])    1.230714
              

                  SimpleBeta
Equity(0 [2330])     1.33995
                  SimpleBeta
Equity(0 [2330])    1.332973
                  SimpleBeta
Equity(0 [2330])    1.351526
                  SimpleBeta
Equity(0 [2330])    1.391541
                  SimpleBeta
Equity(0 [2330])     1.35244
                  SimpleBeta
Equity(0 [2330])    1.383495
                  SimpleBeta
Equity(0 [2330])    1.391533
                  SimpleBeta
Equity(0 [2330])    1.415159
                  SimpleBeta
Equity(0 [2330])    1.417084
                  SimpleBeta
Equity(0 [2330])    1.415846
                  SimpleBeta
Equity(0 [2330])    1.391718
                  SimpleBeta
Equity(0 [2330])    1.405522
                  SimpleBeta
Equity(0 [2330])    1.418297
                  SimpleBeta
Equity(0 [2330])    1.426597
                  SimpleBeta
Equity(0 [2330])    1.433857
                  SimpleBeta
Equity(0 [2330])    1.465513
                  SimpleBeta
Equity(0 [2330])    1.449751
              

                  SimpleBeta
Equity(0 [2330])    0.957006
                  SimpleBeta
Equity(0 [2330])    0.989552
                  SimpleBeta
Equity(0 [2330])    0.976827
                  SimpleBeta
Equity(0 [2330])    1.002162
                  SimpleBeta
Equity(0 [2330])    1.005122
                  SimpleBeta
Equity(0 [2330])    1.042235
                  SimpleBeta
Equity(0 [2330])    1.087289
                  SimpleBeta
Equity(0 [2330])    1.102905
                  SimpleBeta
Equity(0 [2330])     1.08144
                  SimpleBeta
Equity(0 [2330])    1.078982
                  SimpleBeta
Equity(0 [2330])    1.067067
                  SimpleBeta
Equity(0 [2330])    1.087512
                  SimpleBeta
Equity(0 [2330])    1.035889
                  SimpleBeta
Equity(0 [2330])    1.002443
                  SimpleBeta
Equity(0 [2330])    0.952811
                  SimpleBeta
Equity(0 [2330])    0.952717
                  SimpleBeta
Equity(0 [2330])    0.957669
              

                  SimpleBeta
Equity(0 [2330])    1.240273
                  SimpleBeta
Equity(0 [2330])    1.223296
                  SimpleBeta
Equity(0 [2330])    1.244067
                  SimpleBeta
Equity(0 [2330])    1.268328
                  SimpleBeta
Equity(0 [2330])    1.277596
                  SimpleBeta
Equity(0 [2330])     1.28159
                  SimpleBeta
Equity(0 [2330])    1.272632
                  SimpleBeta
Equity(0 [2330])    1.258205
                  SimpleBeta
Equity(0 [2330])    1.296067
                  SimpleBeta
Equity(0 [2330])    1.314551
                  SimpleBeta
Equity(0 [2330])    1.341774
                  SimpleBeta
Equity(0 [2330])    1.356408
                  SimpleBeta
Equity(0 [2330])    1.372011
                  SimpleBeta
Equity(0 [2330])    1.368026
                  SimpleBeta
Equity(0 [2330])    1.365122
                  SimpleBeta
Equity(0 [2330])    1.441132
                  SimpleBeta
Equity(0 [2330])    1.443089
              

<span id="RSI"></span>

### zipline.pipeline.factors.RSI

以 n 天為窗格，計算 RSI。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的價格資訊，預設 = EquityPricing.close。
* window_lengthL: _int_
        以 n 天為週期，預設為 15 日。


[回到選單](#menu)

In [18]:
from zipline.pipeline.factors import RSI
def make_pipeline():
    return Pipeline(
        columns = {
            "RSI": RSI(
                inputs = [TWEquityPricing.close],
                window_length = 87
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

RSI
2018-02-02 00:00:00+00:00 Equity(0 [2330])    60.436893
                          Equity(1 [IR0001])  60.170946
2018-02-05 00:00:00+00:00 Equity(0 [2330])    60.436893
                          Equity(1 [IR0001])  59.138719
2018-02-06 00:00:00+00:00 Equity(0 [2330])    57.793765
...                                                 ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  41.305609
2022-06-30 00:00:00+00:00 Equity(0 [2330])    40.030904
                          Equity(1 [IR0001])  41.586548
2022-07-01 00:00:00+00:00 Equity(0 [2330])    39.244412
                          Equity(1 [IR0001])  40.149446

[2146 rows x 1 columns]

<span id="VWAP"></span>

### zipline.pipeline.factors.VWAP

以 n 天為窗格，計算 Volume Weighted Average Price (以成交量加權價格)。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的價格與成交量資訊，預設 = EquityPricing.close,EquityPricing.volume。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [19]:
from zipline.pipeline.factors import VWAP
def make_pipeline():
    return Pipeline(
        columns = {
            "VWAP": VWAP(
                inputs = [TWEquityPricing.close, TWEquityPricing.volume],
                window_length = 87
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

VWAP
2018-02-02 00:00:00+00:00 Equity(0 [2330])      238.161039
                          Equity(1 [IR0001])  18491.679435
2018-02-05 00:00:00+00:00 Equity(0 [2330])      238.608356
                          Equity(1 [IR0001])  18499.878408
2018-02-06 00:00:00+00:00 Equity(0 [2330])      238.922956
...                                                    ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])  33489.451784
2022-06-30 00:00:00+00:00 Equity(0 [2330])      551.037441
                          Equity(1 [IR0001])  33419.285813
2022-07-01 00:00:00+00:00 Equity(0 [2330])      549.093145
                          Equity(1 [IR0001])  33317.315573

[2146 rows x 1 columns]

<span id="WeightedAverageValue"></span>

### zipline.pipeline.factors.WeightedAverageValue

以 n 天為窗格，與 VWAP 相似，計算以成交量嘉全的某數值平均。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算報酬率所需要的資訊，預設 = EquityPricing.close,EquityPricing.volume。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [20]:
from zipline.pipeline.factors import WeightedAverageValue
def make_pipeline():
    return Pipeline(
        columns = {
            "WeightedAverageValue": WeightedAverageValue(
                inputs = [TWEquityPricing.high, TWEquityPricing.volume],
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

WeightedAverageValue
2018-02-02 00:00:00+00:00 Equity(0 [2330])              260.303304
                          Equity(1 [IR0001])              0.000000
2018-02-05 00:00:00+00:00 Equity(0 [2330])              260.971786
                          Equity(1 [IR0001])              0.000000
2018-02-06 00:00:00+00:00 Equity(0 [2330])              260.065718
...                                                            ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])              0.000000
2022-06-30 00:00:00+00:00 Equity(0 [2330])              501.895564
                          Equity(1 [IR0001])              0.000000
2022-07-01 00:00:00+00:00 Equity(0 [2330])              498.581515
                          Equity(1 [IR0001])              0.000000

[2146 rows x 1 columns]

<span id="PercentChange"></span>

### zipline.pipeline.factors.PercentChange

以 n 天為窗格，計算 x 變數百分比變化。計算公式:

$$
\frac{new - old}{|old|}
$$

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        x 變數資訊。
* window_lengthL: _int_
        以 n 天為週期，需大於等於 2。


[回到選單](#menu)

In [21]:
from zipline.pipeline.factors import PercentChange
def make_pipeline():
    return Pipeline(
        columns = {
            "PercentChange": PercentChange(
                inputs = [TWEquityPricing.close],
                window_length = 2
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

PercentChange
2018-02-02 00:00:00+00:00 Equity(0 [2330])         0.017647
                          Equity(1 [IR0001])       0.005086
2018-02-05 00:00:00+00:00 Equity(0 [2330])         0.000000
                          Equity(1 [IR0001])      -0.003049
2018-02-06 00:00:00+00:00 Equity(0 [2330])        -0.025048
...                                                     ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])      -0.005615
2022-06-30 00:00:00+00:00 Equity(0 [2330])        -0.013065
                          Equity(1 [IR0001])      -0.009731
2022-07-01 00:00:00+00:00 Equity(0 [2330])        -0.030550
                          Equity(1 [IR0001])      -0.025612

[2146 rows x 1 columns]

<span id="PeerCount"></span>

### zipline.pipeline.factors.PeerCount

以 factor/classifier 為判斷基準，呈現同類的公司數量。這裡以報酬率的四分位數為基準。

#### Parameters:
* inputs: _zipline.pipeline.factors.factor_
        判斷基準
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [22]:
from zipline.pipeline.factors import PeerCount, Returns
def make_pipeline():

    Ret = Returns(inputs = [TWEquityPricing.close], window_length = 2)
    quarter = Ret.quartiles()
    
    return Pipeline(
        columns = {
            "PeerCount": PeerCount(
                inputs = [quarter]
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

PeerCount
2018-02-02 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0
2018-02-05 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0
2018-02-06 00:00:00+00:00 Equity(0 [2330])          1.0
...                                                 ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])        1.0
2022-06-30 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0
2022-07-01 00:00:00+00:00 Equity(0 [2330])          1.0
                          Equity(1 [IR0001])        1.0

[2146 rows x 1 columns]

<span id="RateOfChangePercentage"></span>

### zipline.pipeline.factors.RateOfChangePercentage

計算因子在指定時間長度的 %變化量，公式：(尾 - 頭) / 頭 * 100

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [23]:
from zipline.pipeline.factors import RateOfChangePercentage
def make_pipeline():
    
    return Pipeline(
        columns = {
            "RateOfChangePercentage": RateOfChangePercentage(
                inputs = [TWEquityPricing.close],
                window_length = 10
            )
        }
    )
run_pipeline(make_pipeline(), start_time, end_time)

RateOfChangePercentage
2018-02-02 00:00:00+00:00 Equity(0 [2330])                  1.565558
                          Equity(1 [IR0001])                0.084375
2018-02-05 00:00:00+00:00 Equity(0 [2330])                 -0.764818
                          Equity(1 [IR0001])               -0.936954
2018-02-06 00:00:00+00:00 Equity(0 [2330])                 -4.887218
...                                                              ...
2022-06-29 00:00:00+00:00 Equity(1 [IR0001])               -2.510118
2022-06-30 00:00:00+00:00 Equity(0 [2330])                 -3.346457
                          Equity(1 [IR0001])               -2.663798
2022-07-01 00:00:00+00:00 Equity(0 [2330])                 -4.990020
                          Equity(1 [IR0001])               -3.973330

[2146 rows x 1 columns]

<span id="Aroon"></span>

### zipline.pipeline.factors.Aroon

阿隆指標，計算從指定期間 (window_length) 股價最高/低點到現在的時間，回傳一組 (down, up)，數值介於 0 - 100。

計算方式：<br>
up = 取期間 EquityPricing.high 最高那日index，除以 (window_length - 1)，再乘上100 <br>
down = 取期間 EquityPricing.low 最低那日index，除以 (window_length - 1)，再乘上100 <br>
100 = 期間最後一日達到最高/低，0 = 期間第一日達到最高/低<br>

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.low, EquityPricing.high。
* window_lengthL: _int_
        以 n 天為週期。


[回到選單](#menu)

In [24]:
from zipline.pipeline.factors import Aroon
from zipline.pipeline.filters import StaticSids
def make_pipeline():
    
    return Pipeline(
        columns = {
            "Aroon": Aroon(
                inputs = [TWEquityPricing.high, TWEquityPricing.low],
                window_length = 10,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

,,Aroon
2018-02-02 00:00:00+00:00,Equity(0 [2330]),"(0.0, 22.22222222222222)"
2018-02-05 00:00:00+00:00,Equity(0 [2330]),"(44.44444444444444, 11.11111111111111)"
2018-02-06 00:00:00+00:00,Equity(0 [2330]),"(100.0, 0.0)"
2018-02-07 00:00:00+00:00,Equity(0 [2330]),"(100.0, 66.66666666666667)"
2018-02-08 00:00:00+00:00,Equity(0 [2330]),"(88.88888888888889, 55.55555555555556)"
...,...,...
2022-06-27 00:00:00+00:00,Equity(0 [2330]),"(100.0, 0.0)"
2022-06-28 00:00:00+00:00,Equity(0 [2330]),"(88.88888888888889, 22.22222222222222)"
2022-06-29 00:00:00+00:00,Equity(0 [2330]),"(77.77777777777777, 11.11111111111111)"
2022-06-30 00:00:00+00:00,Equity(0 [2330]),"(66.66666666666667, 0.0)"


<span id="FastStochasticOscillator"></span>

### zipline.pipeline.factors.FastStochasticOscillator

快速隨機指標 (K值)，0-100，通常超過 80 有超買跡象，低於 20 有超賣跡象；D 值通常為 K 值三日 SMA。

計算方式：
(最後收盤價 - 期間最低) / (期間最高 - 期間最低) * 100

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.close, EquityPricing.low, EquityPricing.high。
* window_lengthL: _int_
        以 n 天為週期，預設 = 14。

[回到選單](#menu)

In [25]:
from zipline.pipeline.factors import FastStochasticOscillator
def make_pipeline():
    
    return Pipeline(
        columns = {
            "FastStochasticOscillator": FastStochasticOscillator(
                inputs = [TWEquityPricing.close, TWEquityPricing.low, TWEquityPricing.high],
                window_length = 10,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

,,FastStochasticOscillator
2018-02-02 00:00:00+00:00,Equity(0 [2330]),55.172414
2018-02-05 00:00:00+00:00,Equity(0 [2330]),53.571429
2018-02-06 00:00:00+00:00,Equity(0 [2330]),16.129032
2018-02-07 00:00:00+00:00,Equity(0 [2330]),16.666667
2018-02-08 00:00:00+00:00,Equity(0 [2330]),20.000000
...,...,...
2022-06-27 00:00:00+00:00,Equity(0 [2330]),4.808313
2022-06-28 00:00:00+00:00,Equity(0 [2330]),43.548387
2022-06-29 00:00:00+00:00,Equity(0 [2330]),40.322581
2022-06-30 00:00:00+00:00,Equity(0 [2330]),19.354839


<span id="TrueRange"></span>

### zipline.pipeline.factors.TrueRange

真實波動幅度，從
    
    今天最高 - 今天最低
    |今天最高 - 昨天收盤| 
    |今天最低 - 昨天收盤| 
    
中挑出最大值，數值越大代表波動性越大。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.close, EquityPricing.low, EquityPricing.high。
* window_lengthL: _int_
        以 n 天為週期，預設 = 2。

[回到選單](#menu)

In [26]:
from zipline.pipeline.factors import TrueRange
def make_pipeline():
    
    return Pipeline(
        columns = {
            "TrueRange": TrueRange(
                inputs = [TWEquityPricing.close, TWEquityPricing.low, TWEquityPricing.high],
                window_length = 2,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

,,TrueRange
2018-02-02 00:00:00+00:00,Equity(0 [2330]),2.5
2018-02-05 00:00:00+00:00,Equity(0 [2330]),6.0
2018-02-06 00:00:00+00:00,Equity(0 [2330]),9.5
2018-02-07 00:00:00+00:00,Equity(0 [2330]),20.5
2018-02-08 00:00:00+00:00,Equity(0 [2330]),4.0
...,...,...
2022-06-27 00:00:00+00:00,Equity(0 [2330]),8.0
2022-06-28 00:00:00+00:00,Equity(0 [2330]),6.0
2022-06-29 00:00:00+00:00,Equity(0 [2330]),10.0
2022-06-30 00:00:00+00:00,Equity(0 [2330]),9.0


<span id="IchimokuKinkoHyo"></span>

### zipline.pipeline.factors.IchimokuKinkoHyo

輸出順序為 "tenkan_sen" 轉換線， "kijun_sen" 基準線， "senkou_span_a" 先行帶A， "senkou_span_b" 先行帶B， "chikou_span" 遲行帶<br>

"tenkan_sen" 轉換線：(9日最高 ＋ 9日最低) / 2<br>
"kijun_sen" 基準線：(26日最高 ＋ 26日最低) / 2<br>
"senkou_span_a" 先行帶A：前兩項平均<br>
"senkou_span_b" 先行帶B：(52日最高 ＋ 52日最低) / 2<br>
 "chikou_span" 遲行帶：26天前收盤價<br>

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.BoundColumn_
        計算所需價量資料，預設 = EquityPricing.high, EquityPricing.low, EquityPricing.close。
* window_lengthL: _int_
        以 n 天為週期，預設 = 52。
* tenkan_sen_length: _int_
        決定轉換線日數。
* kijun_sen_length: _int_
        決定基準線日數。
* chikou_span_length: _int_
        決定遲行帶日數。

[回到選單](#menu)

In [27]:
from zipline.pipeline.factors import IchimokuKinkoHyo
def make_pipeline():
    
    return Pipeline(
        columns = {
            "IchimokuKinkoHyo": IchimokuKinkoHyo(
                inputs = [TWEquityPricing.high, TWEquityPricing.low, TWEquityPricing.close],
                window_length = 52,
                mask = StaticSids([0])
            )
        }, screen=StaticSids([0])
    )
run_pipeline(make_pipeline(), start_time, end_time)

,,IchimokuKinkoHyo
2018-02-02 00:00:00+00:00,Equity(0 [2330]),"(259.0, 245.5, 252.25, 244.25, 225.0)"
2018-02-05 00:00:00+00:00,Equity(0 [2330]),"(259.0, 245.5, 252.25, 244.25, 226.0)"
2018-02-06 00:00:00+00:00,Equity(0 [2330]),"(257.25, 246.25, 251.75, 244.25, 229.5)"
2018-02-07 00:00:00+00:00,Equity(0 [2330]),"(249.0, 248.5, 248.75, 244.25, 232.5)"
2018-02-08 00:00:00+00:00,Equity(0 [2330]),"(247.75, 249.25, 248.5, 244.25, 237.0)"
...,...,...
2022-06-27 00:00:00+00:00,Equity(0 [2330]),"(500.5, 520.9872298624755, 510.74361493123774,..."
2022-06-28 00:00:00+00:00,Equity(0 [2330]),"(500.5, 520.9872298624755, 510.74361493123774,..."
2022-06-29 00:00:00+00:00,Equity(0 [2330]),"(500.5, 520.9872298624755, 510.74361493123774,..."
2022-06-30 00:00:00+00:00,Equity(0 [2330]),"(495.5, 520.9872298624755, 508.24361493123774,..."
